In [1]:
import set_path

import numpy as np
import pandas as pd
import logging
import time
import glob

import src.utils.ahn_utils as ahn_utils
import src.utils.bgt_utils as bgt_utils
import src.utils.las_utils as las_utils
import src.utils.log_utils as log_utils
import src.utils.plot_utils as plot_utils
import src.analysis.analysis_tools as analysis_tools

from cable_extractor import CableExtractor
from milestone_1.searchspace_reducer import SearchSpaceReducer

log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

logger = logging.getLogger(__name__)

**Load Data**

In [ ]:
# Load data.
tilecode = '2387_9699'

# Labelled LAS file (see Notebook 0).
las_file = '../datasets/Lairessestraat/pointcloud/labelled_' + tilecode + '.laz'
processed_las_file = '../datasets/Lairessestraat/pointcloud/processed_' + tilecode + '.laz'


# AHNReader for elevation data.
ahn_data_folder = '../datasets/Lairessestraat/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)
ahn_tile = ahn_reader.filter_tile(tilecode)

# BGTReader
bgt_building_file = '../datasets/Lairessestraat/bgt/bgt_buildings.csv'
bgt_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_building_file)

In [ ]:
# Load pointcloud
pointcloud = las_utils.read_las(las_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

# colors
pc_colors =  np.vstack((pointcloud.red, pointcloud.blue, pointcloud.green)).T

# Mask
mask = np.ones(len(points), dtype=bool)

In [ ]:
# Load pointcloud
pointcloud = las_utils.read_las(processed_las_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

# colors
pc_colors =  np.vstack((pointcloud.red, pointcloud.blue, pointcloud.green)).T

# Mask
mask = np.ones(len(points), dtype=bool)

**Search Space Reduction** (optional)

In [ ]:
SSR = SearchSpaceReducer(0, ahn_tile, bgt_reader)
mask, labels = SSR.get_label_mask(points, None, None, tilecode)

**Cable Extraction**

In [ ]:
grow_params={'grow_length':1, 'section_length':3, 'cable_width_buffer':0.1}
merge_params={'max_angle_A':45, 'max_dist_A':.5, 'max_angle_B':5, 'max_dist_B':5}

cable_extractor = CableExtractor(11, neighborhood_method='radius', min_points=4, grow_params=grow_params, merge_params=merge_params)

In [ ]:
label_mask = cable_extractor.get_label_mask(points, None, label_mask, tilecode)

**Plot Results**

In [ ]:
plot_utils.plot_pointcloud_o3d(points[label_mask], np.zeros(label_mask.sum()))

#### Batch Folder + Stats

In [2]:
def process_file(in_file, grow_params, merge_params, folder_path, save_output):
        
    tilecode = las_utils.get_tilecode_from_filename(in_file)
    logger.info(f'Processing file {in_file}.')
    start = time.time()
    
    pointcloud = las_utils.read_las(in_file)
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    true_labels = np.zeros((len(points),), dtype='uint16')
    true_labels[(10 < pointcloud.label) & (pointcloud.label < 15)] = 11
    true_mask = true_labels == 11

    pointcloud_labelled = las_utils.read_las(folder_path+'/processed/M1_'+tilecode+'.laz')
    mask = (pointcloud_labelled.label == 0)

    cable_extractor = CableExtractor(11, neighborhood_method='radius', min_points=4, grow_params=grow_params, merge_params=merge_params)
    label_mask = cable_extractor.get_label_mask(points, None, mask, tilecode)
    
    labels_pred = np.zeros((len(points),), dtype='uint16')
    labels_pred[label_mask] = 11

    duration = time.time() - start
    stats, report = analysis_tools.get_cable_stats_m2(labels_pred, true_mask, mask)
    report['time'] = duration
    report['tilecode'] = tilecode

    logger.info('STATISTICS\n' + stats)
    logger.info(f'File processed in {duration:.2f}s.')

    if save_output:
        outpath = folder_path + '/processed/M2_' + tilecode + '.laz'
        las_utils.label_and_save_las(pointcloud, labels_pred, outpath)

    return report

    

In [6]:
def process_folder(folder_path, save_output=False):

    # Parameters
    grow_params={'grow_length':1, 'section_length':3, 'cable_width_buffer':0.1}
    merge_params={'max_angle_A':45, 'max_dist_A':.5, 'max_angle_B':5, 'max_dist_B':5}

    results = []

    for file in glob.iglob(folder_path + '/pointcloud/labelled_*.laz'):
        report = process_file(file, grow_params, merge_params, folder_path, save_output)
        results.append(report)

    return pd.DataFrame(results)



In [10]:
df = process_folder('../datasets/Lairessestraat', True)

INFO - 2022-04-26 11:07:59,764 - 2269192256 - Processing file ../datasets/Lairessestraat/pointcloud/labelled_2394_9702.laz.
INFO - 2022-04-26 11:08:03,050 - cable_extractor - Cable Extractor (label=11)
/opt/anaconda3/envs/pointcloud/lib/python3.8/site-packages/pyransac3d/line.py:47: RuntimeWarning: invalid value encountered in true_divide
  vecA_norm = vecA / np.linalg.norm(vecA)
/Users/FalkeBoskaljon/Documents/Study/Master/Internship/Urban_PointCloud_Processing/milestone_2/cable_extractor.py:207: RuntimeWarning: invalid value encountered in long_scalars
  gap_score = np.sum(gap_inliers) / len(gap_pts)
/opt/anaconda3/envs/pointcloud/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/FalkeBoskaljon/Documents/Study/Master/Internship/Urban_PointCloud_Processing/milestone_2/cloud_utils.py:21: RuntimeWarning: invalid value encountered in 

In [11]:
df

,start_pts,end_pts,reduce_per,cable_pts,recall,precision,f1_score,time,tilecode
0,186711,8769,0.95,22403,1.0,0.0,0.0,71.533692,2394_9702
1,491075,21017,0.96,21777,1.0,0.0,0.0,166.068013,2388_9700
2,508411,12624,0.98,13113,1.0,0.0,0.0,134.111967,2389_9700
3,162858,4992,0.97,6434,1.0,0.0,0.0,34.479340,2391_9701
4,151087,7449,0.95,7654,1.0,0.0,0.0,49.700665,2386_9699
5,92320,23011,0.75,23948,1.0,0.0,0.0,80.313083,2387_9699
6,509014,4253,0.99,4085,1.0,0.0,0.0,119.538043,2387_9700


,tilecode,recall,precision,f1_score
0,2394_9702,0.388787,0.993272,0.558835
1,2388_9700,0.955733,0.990294,0.972706
2,2389_9700,0.955235,0.992237,0.973385
3,2391_9701,0.774479,0.998197,0.872221
4,2386_9699,0.969689,0.996375,0.982851
5,2387_9699,0.918574,0.955978,0.936902
6,2387_9700,0.990942,0.951799,0.970976
